In [102]:
import sqlite3 as sql
import pandas as pd 

Apuntador = sql.connect(r'..\DataBase\DataUsers.db')

In [103]:
hd = pd.read_sql_query("SELECT * FROM Aula_hki620", Apuntador)

In [104]:
hd

,Id_E,Nombre_Estudiante,Correo
0,None,Angel Arevalo,@UNAL.EDU.CO


In [40]:
query_table = "SELECT name FROM sqlite_master WHERE type='table' AND name = 'Aula_tju180'"
result = Apuntador.execute(query_table)
result = result.fetchone()
result = result[0] if (result != None) else result
print(result)

Aula_tju180


In [11]:
df = pd.read_sql_query("SELECT Usuarios_Registrados.Nombre_Usuario, Roles.Rol FROM Usuarios_Registrados JOIN Roles ON Usuarios_Registrados.Rol = Roles.Id_Rol", Apuntador)

In [5]:
b = Apuntador.execute("PRAGMA table_info('Usuarios_Registrados')")
b = bool(b.fetchall())
print(b)

True


In [4]:
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", Apuntador)

In [37]:
b = Apuntador.execute("SELECT name FROM sqlite_master WHERE type='table'")
b = b.fetchall()
for i in range(len(b)):
    Apuntador.execute(f"DROP TABLE '{b[i][0]}'")

#CUIDADO Elimina todas las tablas

OperationalError: database is locked

In [6]:
df = pd.read_sql_query("SELECT * FROM 'Aula_qjl-030';", Apuntador)

In [96]:
import sqlite3

conn = sqlite3.connect(r'..\DataBase\DataUsers.db')

cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cur.fetchall()

for table in tables:
    cur.execute("DROP TABLE IF EXISTS " + table[0])

conn.commit()

cur.close()
conn.close()

print("All data in the database has been deleted.")

All data in the database has been deleted.


In [84]:
import sqlite3

conn = sqlite3.connect(r'..\DataBase\DataUsers.db')

dc = conn.cursor()

df = dc.execute("SELECT * FROM 'User_Aulas'")

rows = dc.fetchall()

print("Usuarios_Registrados table:")
for row in rows:
    print(row)

conn.commit()
conn.close()

Usuarios_Registrados table:
(1, 2, 1)


In [8]:
df

In [105]:
Apuntador.commit()
Apuntador.close()